In [ ]:
!pip install torch torchvision opencv-python numpy matplotlib h5py

In [ ]:
import torch
from model import CSRNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CSRNet().to(device)

# Safely load full checkpoint file
checkpoint = torch.load("partBmodel_best.pth", map_location=device, weights_only=False)

# Check what's inside
if 'state_dict' in checkpoint:
    print("✔ Found 'state_dict' in checkpoint — loading it...")
    model.load_state_dict(checkpoint['state_dict'])
else:
    print("⚠ No 'state_dict' found — assuming raw weights dict...")
    model.load_state_dict(checkpoint)

model.eval()

In [ ]:
!pip install tqdm

In [ ]:
import tqdm

In [ ]:
!pip install ipywidgets tqdm --upgrade

In [ ]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import tqdm
from tqdm import tqdm


# === 1. Set device and load model ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CSRNet().to(device)

# Load checkpoint (adjust path if needed)
checkpoint = torch.load("partBmodel_best.pth", map_location=device, weights_only=False)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

# === 2. Preprocessing for each frame ===
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# === 3. Load Video ===
input_path = r"C:\Users\shrut\Downloads\Video\trim\trim3.mp4"
# output_path = r"C:\Users\shrut\Downloads\new_leeyeehoo_model\output_trim4.mp4"

cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print("Processing video...")
for _ in tqdm(range(frame_count)):
    ret, frame = cap.read()
    if not ret:
        break

    # === 4. Frame Preprocessing ===
    # img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # img_pil = Image.fromarray(img_rgb)
    # img_tensor = transform(img_pil).unsqueeze(0).to(device)

    # === 5. Forward pass ===
    # with torch.no_grad():
    #     density_map = model(img_tensor)
    #     density_np = density_map.squeeze().cpu().numpy()
    #     count = int(density_np.sum())

    def count_frame_patched(frame, model, transform, patch_size=(384, 384)):
    H, W, _ = frame.shape
    patch_h, patch_w = patch_size
    total_count = 0

    for y in range(0, H, patch_h):
        for x in range(0, W, patch_w):
            patch = frame[y:min(y+patch_h, H), x:min(x+patch_w, W)]
            patch_resized = cv2.resize(patch, patch_size)
            patch_rgb = cv2.cvtColor(patch_resized, cv2.COLOR_BGR2RGB)
            patch_pil = Image.fromarray(patch_rgb)
            input_tensor = transform(patch_pil).unsqueeze(0).to(device)

            with torch.no_grad():
                density_map = model(input_tensor)
                count = float(density_map.sum().item())
                total_count += count

    return int(total_count)

    # === 5. Patched forward pass ===
    count = count_frame_patched(frame, model, transform)



    # === 6. Draw count ===
    cv2.putText(frame, f"Count: {count}", (30, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 0, 255), 3)

    # === 7. Live display + write output ===
    cv2.imshow('Live Headcount', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Press Esc to stop early
        break

    # out.write(frame)


# === 8. Cleanup ===
cap.release()
# out.release()
# print("✅ Video saved at:", output_path)
cv2.destroyAllWindows()



In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

# === 1. Define CSRNet model (reuse your existing model.py) ===
from model import CSRNet

# === 2. Load model ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CSRNet().to(device)

checkpoint = torch.load("partBmodel_best.pth", map_location=device, weights_only=False)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

# === 3. Define transform ===
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# === 4. Patch-based CSRNet counting ===
def count_frame_patched(frame, model, transform, patch_size=(384, 384)):
    H, W, _ = frame.shape
    patch_h, patch_w = patch_size
    total_count = 0

    for y in range(0, H, patch_h):
        for x in range(0, W, patch_w):
            patch = frame[y:min(y+patch_h, H), x:min(x+patch_w, W)]
            patch_resized = cv2.resize(patch, patch_size)
            patch_rgb = cv2.cvtColor(patch_resized, cv2.COLOR_BGR2RGB)
            patch_pil = Image.fromarray(patch_rgb)
            input_tensor = transform(patch_pil).unsqueeze(0).to(device)

            with torch.no_grad():
                density_map = model(input_tensor)
                count = float(density_map.sum().item())
                total_count += count

    return int(total_count)

# === 5. Load video ===
input_path = r"C:\Users\shrut\Downloads\Video\trim\maingate.mp4"
cap = cv2.VideoCapture(input_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# === 6. Process and display ===
print("🔍 Live processing... Press ESC to quit.")

frame_skip = 5

for i in tqdm(range(0, frame_count, frame_skip)):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        break
# for _ in tqdm(range(frame_count)):
#     ret, frame = cap.read()
#     if not ret:
#         break

    count = count_frame_patched(frame, model, transform)

    # Overlay count
    cv2.putText(frame, f"Count: {count}", (30, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 0, 255), 3)

    # Live display
    cv2.imshow('Live Crowd Headcount (CSRNet)', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()
print("✅ Done.")


In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

# === 1. Define CSRNet model (reuse your existing model.py) ===
from model import CSRNet

# === 2. Load model ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CSRNet().to(device)

checkpoint = torch.load("partBmodel_best.pth", map_location=device, weights_only=False)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

# === 3. Define transform ===
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# === 4. Patch-based CSRNet counting ===
def count_frame_patched(frame, model, transform, patch_size=(384, 384)):
    H, W, _ = frame.shape
    patch_h, patch_w = patch_size
    total_count = 0

    for y in range(0, H, patch_h):
        for x in range(0, W, patch_w):
            patch = frame[y:min(y+patch_h, H), x:min(x+patch_w, W)]
            patch_resized = cv2.resize(patch, patch_size)
            patch_rgb = cv2.cvtColor(patch_resized, cv2.COLOR_BGR2RGB)
            patch_pil = Image.fromarray(patch_rgb)
            input_tensor = transform(patch_pil).unsqueeze(0).to(device)

            with torch.no_grad():
                density_map = model(input_tensor)
                count = float(density_map.sum().item())
                total_count += count

    return int(total_count)

# === 5. Ask user for input source ===
print("🔘 Choose input source:")
print("1: Local video file")
print("2: Live CCTV stream (RTSP link)")
print("3: Webcam")
choice = input("Enter your choice (1/2/3): ").strip()

if choice == "1":
    input_path = input("Enter the full path of the video file: ").strip().strip('"')
elif choice == "2":
    input_path = input("Enter the RTSP link of the CCTV stream: ").strip()
elif choice == "3":
    input_path = 0  # Default webcam
else:
    print("❌ Invalid choice.")
    exit()


cap = cv2.VideoCapture(input_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# === 6. Process and display ===
print("🔍 Live processing... Press ESC to quit.")

frame_skip = 5

for i in tqdm(range(0, frame_count, frame_skip)):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        break


    count = count_frame_patched(frame, model, transform)

    # Overlay count
    cv2.putText(frame, f"Count: {count}", (30, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 0, 255), 3)

    # Live display
    cv2.imshow('Live Crowd Headcount (CSRNet)', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()
print("✅ Done.")